
# <span style="color:red">**Decision Tree Implementation**</span>

#### Reading data From File  ,  Importing Packages , Converting all the values to Float from String

In [1]:
import csv
import os
import numpy as np
from math import log
os.chdir("C:/Users/Satya/Files")
# Load data set
with open("wine-dataset.csv") as f:
    next(f, None)
    data = [list(line) for line in csv.reader(f, delimiter=",")]
data =np.array(data,float)
print ("Number of records: %d" % len(data))

Number of records: 4898


#### Splitting Data into Train-Test  ( 75%  Train &  25% Test )

In [2]:
    # Split training/test sets
    # You need to modify the following code for cross validation.
    K = 4
    training_set = [x for i, x in enumerate(data) if i % K != 3]
    test_set = [x for i, x in enumerate(data) if i % K == 3]

###### Creating Function to bin Feature Values to create multiple thresholds for finding the best condition for Information Gain

In [3]:
def binned_vals(rows, col):
    values= set([row[col] for row in rows])
    return list(np.histogram([float(i) for i in values],bins=10)[1])

##### Counting class labels for a dataset or subset of data arriving at a Node

In [4]:
def class_counts(rows):
    counts = {}  
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

#### Function for Calculating Entropy Calculation

In [5]:
def Entropy_Calc(rows):
    counts = class_counts(rows)
    entropy = 0
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        entropy -= prob_of_lbl * log(prob_of_lbl, 2)
    return entropy

#### Function for Creating Evaluation Criterias for various theresholds for Any Feature

In [6]:
class Eval_Criteria:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def Eval(self, example):
        val = example[self.column]
        return val >= self.value

####  Function for binary univariate split for the  Data Set / Subset of Data  at a Decision Node

In [7]:
def Splits(rows, Evals_Criteria):
    right_rows, left_rows = [], []
    for row in rows:
        if Evals_Criteria.Eval(row):
            right_rows.append(row)
        else:
            left_rows.append(row)
    return right_rows, left_rows

##### Function for Calculating Information Gain 

In [8]:
def info_gain(left, right, current_entropy):
    p = float(len(left)) / (len(left) + len(right))
    return current_entropy - p * Entropy_Calc(left) - (1 - p) * Entropy_Calc(right)

#### Function for Finding Best Split or Evaluation Criteria  at each node by calculating Information Gain

In [9]:
def find_best_split(rows):
    best_gain = 0  
    best_eval_criteria = None  
    current_entropy = Entropy_Calc(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  
        values = binned_vals(rows,col) 

        for val in values:  # for each value
            Evals_Criteria = Eval_Criteria(col, val)
            right_rows, left_rows = Splits(rows, Evals_Criteria)
            
            if len(right_rows) == 0 or len(left_rows) == 0:
                continue

            gain = info_gain(right_rows, left_rows, current_entropy)

            if gain >= best_gain:
                best_gain, best_eval_criteria = gain, Evals_Criteria

    return best_gain, best_eval_criteria

#### Class for Prediction Node

In [10]:
class Prediction_Node:
    def __init__(self, rows):
        self.predictions = float(list(str(list(str(class_counts(rows).keys()).split('['))[1]).split(']'))[0])

#### Class for Decision Node

In [11]:
class Decision_Node:
    def __init__(self,
                 Evals_Criteria,
                 true_branch,
                 false_branch):
        self.Evals_Criteria = Evals_Criteria
        self.true_branch = true_branch
        self.false_branch = false_branch

#### Function for Building the Tree recursively

In [12]:
def build_tree(rows):
    gain, Evals_Criteria = find_best_split(rows)
    if gain == 0:
        return Prediction_Node(rows)
    true_rows, false_rows = Splits(rows, Evals_Criteria)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(Evals_Criteria, true_branch, false_branch)

#### Calling the Function to  Build the Tree recursively

In [13]:
my_tree = build_tree(training_set)

#### Function for Classifying Test Data

In [14]:
def classify(row, node):
    if isinstance(node, Prediction_Node):
        return node.predictions
    if node.Evals_Criteria.Eval(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [15]:
results= []
for row in test_set:
    result = classify(row, my_tree)
    results.append( result == row[-1])
accuracy = float(results.count(True))/float(len(results))
print ( "The Accuracy for Test set which is  25% of toal Data Set is" , accuracy *100 )

The Accuracy for Test set which is  25% of toal Data Set is 83.16993464052288


### <span style="color:blue">Conclusion</span>

#### <span style="color:blue">Above is the Decision Tree Algorithm .  The Accuracy for Test set which is  25% of toal Data Set is 83.17 % </span>


# <span style="color:red">**Cross-Validation**</span>


In [16]:
    K = 10
    lst = np.arange(1,10)
    accuracy = []
    for j in lst:
        training_set = [x for i, x in enumerate(data) if i % K != j]
        test_set = [x for i, x in enumerate(data) if i % K == j]
        my_tree = build_tree(training_set)
        results= []
        for row in test_set:
            result = classify(row, my_tree)
            results.append( result == row[-1])
        accuracy.append(float(results.count(True))/float(len(results)))
        print ( " The accuracy for " , j , " iteration of Cross-Validation is " , float(results.count(True))/float(len(results))*100 , "%")

 The accuracy for  1  iteration of Cross-Validation is  86.93877551020408 %
 The accuracy for  2  iteration of Cross-Validation is  83.06122448979592 %
 The accuracy for  3  iteration of Cross-Validation is  84.08163265306122 %
 The accuracy for  4  iteration of Cross-Validation is  85.3061224489796 %
 The accuracy for  5  iteration of Cross-Validation is  84.48979591836735 %
 The accuracy for  6  iteration of Cross-Validation is  83.06122448979592 %
 The accuracy for  7  iteration of Cross-Validation is  84.28571428571429 %
 The accuracy for  8  iteration of Cross-Validation is  82.82208588957054 %
 The accuracy for  9  iteration of Cross-Validation is  81.18609406952966 %


In [17]:
print ( "The Average accuracy for Cross-Validation is " , np.average(accuracy)*100 , "%" )

The Average accuracy for Cross-Validation is  83.9147410839 %


### <span style="color:blue">Conclusion</span>

#### <span style="color:blue">Above is the implementation of Cross Validation for 10 Folds .  The Average accuracy for Cross-Validation is  83.92 % </span>


# <span style="color:red">** Improvement Strategies**</span>

### <span style="color:orange"> Strategy 1 :- Using Gini Index Instead of Entropy</span>

#### Function for Calculating Gini Index

In [18]:
def Gini_index_Calc(rows):
    counts = class_counts(rows)
    Gini_index = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        Gini_index -= prob_of_lbl**2
    return Gini_index

##### Function for Calculating Information Gain for Gini Index

In [19]:
def info_gain_gini(left, right, current_Gini_index):
    p = float(len(left)) / (len(left) + len(right))
    return current_Gini_index - p * Gini_index_Calc(left) - (1 - p) * Gini_index_Calc(right)

#### Function for Finding Best Split or Evaluation Criteria  at each node by calculating Information Gain for Gini Index

In [20]:
def find_best_split(rows):
    best_gain = 0  
    best_eval_criteria = None  
    current_Gini_index = Gini_index_Calc(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  
        values = binned_vals(rows,col) 

        for val in values:  # for each value
            Evals_Criteria = Eval_Criteria(col, val)
            right_rows, left_rows = Splits(rows, Evals_Criteria)
            
            if len(right_rows) == 0 or len(left_rows) == 0:
                continue

            gain = info_gain_gini(right_rows, left_rows, current_Gini_index)

            if gain >= best_gain:
                best_gain, best_eval_criteria = gain, Evals_Criteria

    return best_gain, best_eval_criteria

In [21]:
my_tree = build_tree(training_set)

In [22]:
results= []
for row in test_set:
    result = classify(row, my_tree)
    results.append( result == row[-1])

In [23]:
accuracy = float(results.count(True))/float(len(results))
print ( "The Accuracy for Test set which is  25% of toal Data Set is" , accuracy *100 )

The Accuracy for Test set which is  25% of toal Data Set is 82.0040899795501


In [25]:
    K = 10
    lst = np.arange(1,10)
    accuracy = []
    for j in lst:
        training_set = [x for i, x in enumerate(data) if i % K != j]
        test_set = [x for i, x in enumerate(data) if i % K == j]
        my_tree = build_tree(training_set)
        results= []
        for row in test_set:
            result = classify(row, my_tree)
            results.append( result == row[-1])
        accuracy.append(float(results.count(True))/float(len(results)))
        print ( " The accuracy for " , j , " iteration of Cross-Validation is " , float(results.count(True))/float(len(results))*100 , "%")

 The accuracy for  1  iteration of Cross-Validation is  84.08163265306122 %
 The accuracy for  2  iteration of Cross-Validation is  83.46938775510205 %
 The accuracy for  3  iteration of Cross-Validation is  83.6734693877551 %
 The accuracy for  4  iteration of Cross-Validation is  86.12244897959184 %
 The accuracy for  5  iteration of Cross-Validation is  83.6734693877551 %
 The accuracy for  6  iteration of Cross-Validation is  84.48979591836735 %
 The accuracy for  7  iteration of Cross-Validation is  83.46938775510205 %
 The accuracy for  8  iteration of Cross-Validation is  84.66257668711657 %
 The accuracy for  9  iteration of Cross-Validation is  82.0040899795501 %


In [26]:
print ( "The Average accuracy for Cross-Validation is " , np.average(accuracy)*100 , "%" )

The Average accuracy for Cross-Validation is  83.9606953893 %


### <span style="color:blue">Conclusion</span>

#### <span style="color:blue">Above is the startegy 1  for improving the tree's performance . We have implemented Gini Index here instead of Entropy as Information Gain Measure.  The Average accuracy for Cross-Validation is  83.96 % </span>

### <span style="color:orange"> Strategy 2 :- Pruning the Tree</span>

#### Function for Building the Tree recursively  & Prune it after 20 Levels

In [27]:
Depth_true = 0
Depth_False = 0

def build_tree_Pruned(rows):
    global Depth_true
    global Depth_False
    gain, Evals_Criteria = find_best_split(rows)
    if gain == 0:
        return Prediction_Node(rows)
    true_rows, false_rows = Splits(rows, Evals_Criteria)
    
    if Depth_true <= 15 :
        true_branch = build_tree(true_rows)
        Depth_true += 1
    else:
        return Prediction_Node(rows)
    
    if Depth_False <= 15 :
        false_branch = build_tree(false_rows)
        Depth_False += 1
    else:
        return Prediction_Node(rows)
    
    return Decision_Node(Evals_Criteria, true_branch, false_branch)

In [28]:
my_tree_pruned = build_tree_Pruned(training_set)

In [29]:
results= []
for row in test_set:
    result = classify(row, my_tree_pruned)
    results.append( result == row[-1])
    
accuracy = float(results.count(True))/float(len(results))
print ( "The Accuracy for Test set which is  25% of toal Data Set is" , accuracy *100 )

The Accuracy for Test set which is  25% of toal Data Set is 82.0040899795501


In [30]:
    K = 10
    lst = np.arange(1,10)
    accuracy = []
    for j in lst:
        training_set = [x for i, x in enumerate(data) if i % K != j]
        test_set = [x for i, x in enumerate(data) if i % K == j]
        my_tree = build_tree_Pruned(training_set)
        results= []
        for row in test_set:
            result = classify(row, my_tree)
            results.append( result == row[-1])
        accuracy.append(float(results.count(True))/float(len(results)))
        print ( " The accuracy for " , j , " iteration of Cross-Validation is " , float(results.count(True))/float(len(results))*100 , "%")

 The accuracy for  1  iteration of Cross-Validation is  84.08163265306122 %
 The accuracy for  2  iteration of Cross-Validation is  83.46938775510205 %
 The accuracy for  3  iteration of Cross-Validation is  83.6734693877551 %
 The accuracy for  4  iteration of Cross-Validation is  86.12244897959184 %
 The accuracy for  5  iteration of Cross-Validation is  83.6734693877551 %
 The accuracy for  6  iteration of Cross-Validation is  84.48979591836735 %
 The accuracy for  7  iteration of Cross-Validation is  83.46938775510205 %
 The accuracy for  8  iteration of Cross-Validation is  84.66257668711657 %
 The accuracy for  9  iteration of Cross-Validation is  82.0040899795501 %


In [32]:
print ( "The Average accuracy for Cross-Validation is " , np.average(accuracy)*100 , "%" )

The Average accuracy for Cross-Validation is  83.9606953893 %


### <span style="color:blue">Conclusion</span>

#### <span style="color:blue">Above is the startegy 2 for improving the tree's performance . We have implemented pruning of the tree here where the depth of the tree doesn't  go beyond 15 .  The Average accuracy for Cross-Validation is  83.96 %  </span>